In [1]:
import torch
import torchvision
from torchinfo import summary
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import os
import random
import numpy as np
from tqdm.auto import tqdm

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.metrics import AverageMeter, ProgressMeter
from going_modular.train_eval.train_id_acc import train_id_accuracy

from sklearn.metrics import roc_curve, auc
import torch.nn.functional as F
import numpy as np
device = "cuda" if torch.cuda.is_available() else "cpu"

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
CONFIGURATION = {
    # Thư mục
    'type': 'normalmap',
    'train_dir': './Dataset/Normal_Map/train',
    'test_dir': './Dataset/Normal_Map/test',
    
    # Cấu hình train
    'epochs': 2000,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('./Dataset/Normal_Map/train')),
    'embedding_size': 512,
    
    'learning_rate': 0.1,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

# Dataloader

In [3]:
# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [4]:
# train_transform = transforms.Compose([
#     transforms.RandomCrop(224),
# ])

# val_transform = transforms.Compose([
#     transforms.CenterCrop(224),
# ])

In [5]:
train_dataloader, val_dataloader = create_magface_dataloader(CONFIGURATION, None, None)

In [6]:
dataiter = iter(train_dataloader)
images, labels = next(dataiter)
print(f"Labels: {labels}")

Labels: tensor([ 84,  66, 131,  69,  96, 114,  56, 145, 149, 129, 110, 190,  25,  81,
        195,   8])


# Train loop

In [7]:
model = MagFaceRecognition(CONFIGURATION).to(device)
optimizer = torch.optim.SGD(
        filter(lambda p: p.requires_grad, model.parameters()),
        CONFIGURATION['learning_rate'],
        momentum=CONFIGURATION['momentum'],
        weight_decay=CONFIGURATION['weight_decay']
)
# optimizer = torch.optim.SGD(
#         filter(lambda p: p.requires_grad, model.parameters()),
#         args.lr,
#         momentum=args.momentum,
#         weight_decay=args.weight_decay)
#     pprint.pprint(optimizer)

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-6)

criterion = MagLoss(CONFIGURATION)

In [9]:
def check_ROC(dataloader, model, device):
    model.eval()
    with torch.no_grad():
        outputs_list = []

        for batch in dataloader:
            images, ids = batch
            images = images.to(device)
            outputs = model.get_embedding(images)
            
            for i in range(outputs.size(0)):
                outputs_list.append((ids[i].item(), outputs[i].cpu().numpy()))
                
        scores_dis = []
        labels_dis = []
        scores_cos = []
        labels_cos = []

        for i in range(len(outputs_list)):
            for j in range(i + 1, len(outputs_list)):
                id1, tensor1 = outputs_list[i]
                id2, tensor2 = outputs_list[j]

                # Chuyển đổi numpy array về tensor và chuyển lên thiết bị cuda
                tensor1 = torch.tensor(tensor1)
                tensor2 = torch.tensor(tensor2)

                # Tính khoảng cách Euclidean
                score_dis = F.pairwise_distance(tensor1, tensor2).item()
                # Tính cosine similarity
                score_cos = F.cosine_similarity(tensor1.unsqueeze(0), tensor2.unsqueeze(0)).item()

                # Lưu độ tương đồng và nhãn
                scores_dis.append(score_dis)
                labels_dis.append(0 if id1 == id2 else 1)
                scores_cos.append(score_cos)
                labels_cos.append(1 if id1 == id2 else 0)

        y_true_dis = np.array(labels_dis)
        y_scores_dis = np.array(scores_dis)
        fpr_dis, tpr_dis, thresholds_dis = roc_curve(y_true_dis, y_scores_dis)
        roc_auc_dis = auc(fpr_dis, tpr_dis)

        y_true_cos = np.array(labels_cos)
        y_scores_cos = np.array(scores_cos)
        fpr_cos, tpr_cos, thresholds_cos = roc_curve(y_true_cos, y_scores_cos)
        roc_auc_cos = auc(fpr_cos, tpr_cos)
    return roc_auc_dis, roc_auc_cos

In [10]:
for epoch in range(CONFIGURATION['epochs']):
    train_loss = AverageMeter('Train Loss', ':.3f')
    train_loss_id = AverageMeter('Train Loss Id', ':6.2f')
    train_id_top1 = AverageMeter('Train Top 1 Accuracy', ':6.2f')
    train_id_top5 = AverageMeter('Train Top 5 Accuracy', ':6.2f')
    
    model.train()
    
    for index, (input, target) in enumerate(train_dataloader):
        input = input.to(device)
        target = target.to(device)
        logits, x_norm = model(input)
        
        # caculate metric
        loss_id, loss_g = criterion(logits, target, x_norm)
        loss = loss_id + loss_g
        acc1, acc5 = train_id_accuracy(logits[0], target)
        
        # update metric
        batch_size = input.size(0)
        train_loss.update(loss.item(), batch_size)
        train_loss_id.update(loss_id.item(), batch_size)
        train_id_top1.update(acc1, batch_size)
        train_id_top5.update(acc5, batch_size)
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
    train_loss.compute()
    train_loss_id.compute()
    train_id_top1.compute()
    train_id_top5.compute()
    
    train_roc_dis, train_auc_cos = check_ROC(train_dataloader, model, device)
    
    val_roc_dis, val_auc_cos = check_ROC(val_dataloader, model, device)
    
    train_metrics = [
        train_loss, 
        train_loss_id, 
        train_id_top1 * 100,
        train_id_top5 * 100,
        f"roc_dis: {train_roc_dis:.3f}",
        f"auc_cos: {train_auc_cos:.3f}",
    ]
    
    val_metrics = [
        f"roc_dis: {val_roc_dis:.3f}",
        f"auc_cos: {val_auc_cos:.3f}",
    ]
    
    process = ProgressMeter(
        train_meters=train_metrics,
        val_meters=val_metrics,
        prefix=f"Epoch {epoch + 1}:"
    )
    
    process.display()

Epoch 1:
	train: Train Loss 3.736 | Train Loss Id   3.71 | Train Top 1 Accuracy   0.00 | Train Top 5 Accuracy   0.02 | roc_dis: 0.550 | auc_cos: 0.562
	val: roc_dis: 0.526 | auc_cos: 0.574
Epoch 2:
	train: Train Loss 1.946 | Train Loss Id   1.92 | Train Top 1 Accuracy   0.01 | Train Top 5 Accuracy   0.03 | roc_dis: 0.553 | auc_cos: 0.532
	val: roc_dis: 0.550 | auc_cos: 0.542
Epoch 3:
	train: Train Loss 0.626 | Train Loss Id   0.60 | Train Top 1 Accuracy   0.00 | Train Top 5 Accuracy   0.02 | roc_dis: 0.553 | auc_cos: 0.522
	val: roc_dis: 0.552 | auc_cos: 0.536
Epoch 4:
	train: Train Loss 0.562 | Train Loss Id   0.54 | Train Top 1 Accuracy   0.01 | Train Top 5 Accuracy   0.03 | roc_dis: 0.553 | auc_cos: 0.546
	val: roc_dis: 0.540 | auc_cos: 0.551
Epoch 5:
	train: Train Loss 0.497 | Train Loss Id   0.47 | Train Top 1 Accuracy   0.01 | Train Top 5 Accuracy   0.04 | roc_dis: 0.566 | auc_cos: 0.580
	val: roc_dis: 0.581 | auc_cos: 0.630


KeyboardInterrupt: 

# Test

In [ ]:
# import cv2, os

# os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

In [ ]:
# dataset = './Dataset/Normal_Map/train'

# for id in os.listdir(dataset):
#     id_path = os.path.join(dataset, id)
#     for filename in os.listdir(id_path):
#         image_path = os.path.join(id_path, filename)
#         image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
#         height, width = image.shape[:2]
#         if width != 224 or height != 224:
#             print(image_path)

In [ ]:
# image = cv2.imread('./Dataset/Normal_Map/train/2011/2008-03-27_17-47-12.exr', cv2.IMREAD_UNCHANGED)
# print(image.shape)

(244, 244, 3)
